In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.metrics import r2_score

In [4]:
train_df = pd.read_excel('https://github.com/pykwon/python/blob/master/testdata_utf8/hd_carprice.xlsx?raw=true', sheet_name='train')
test_df = pd.read_excel('https://github.com/pykwon/python/blob/master/testdata_utf8/hd_carprice.xlsx?raw=true', sheet_name='test')
print(train_df.head(2))
print(test_df.head(2))
# feature
x_train = train_df.drop(['가격'], axis=1)
x_test = test_df.drop(['가격'], axis=1)

#label
y_train = train_df[['가격']] 
y_test = test_df[['가격']] 

print(x_train.head(2))
print(x_train.columns)
print(x_train.shape)

print(set(x_train.종류)) # {'소형', '준중형', '중형', '대형'}
print(set(x_train.연료)) # {'가솔린', '디젤', 'LPG'}
print(set(x_train.변속기)) # {'수동', '자동'}

# 종류, 연료, 변속기 열에 대해서는 labelencoder(), onehotencoder() 등을 적용
transformer = make_column_transformer((OneHotEncoder(), ['종류', '연료', '변속기']), remainder='passthrough')

# remainder='passthrouth' 기본은 'drop. 열이 transformer에 전달
transformer.fit(x_train)
x_train = transformer.transform(x_train)  #3개의 칼럼을 포함해 모든 칼럼이 표준화 됨
x_test = transformer.transform(x_test)
print(x_train[:2], x_train.shape) # (71, 16)
print(y_train.shape) # (71, 1)

     가격    년식   종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  1885  2015  준중형  11.8  172  21.0  가솔린      0  1999  1300  자동
1  2190  2015  준중형  12.3  204  27.0  가솔린      0  1591  1300  자동
     가격    년식  종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  1915  2015  대형   6.8  159  23.0  LPG      0  2359  1935  수동
1  1164  2012  소형  13.3  108  13.9  가솔린      0  1396  1035  자동
     년식   종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  2015  준중형  11.8  172  21.0  가솔린      0  1999  1300  자동
1  2015  준중형  12.3  204  27.0  가솔린      0  1591  1300  자동
Index(['년식', '종류', '연비', '마력', '토크', '연료', '하이브리드', '배기량', '중량', '변속기'], dtype='object')
(71, 10)
{'준중형', '대형', '소형', '중형'}
{'가솔린', '디젤', 'LPG'}
{'수동', '자동'}
[[0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 1.000e+00 2.015e+03 1.180e+01 1.720e+02 2.100e+01 0.000e+00
  1.999e+03 1.300e+03]
 [0.000e+00 0.000e+00 1.000e+00 0.000e+00 0.000e+00 1.000e+00 0.000e+00
  0.000e+00 1.000e+00 2.015e+03 1.230e+01 2.040e

In [5]:
# functional api로 모델 작성
from tabnanny import verbose
from matplotlib import units
from scipy import optimize
from sklearn import metrics



input = tf.keras.layers.Input(shape=(16,))
net = tf.keras.layers.Dense(units=32, activation='relu')(input)
net = tf.keras.layers.Dense(units=32, activation='relu')(net)
net = tf.keras.layers.Dense(units=1, activation='relu')(net)

model = tf.keras.models.Model(input, net)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])
print(model.summary)

<bound method Model.summary of <Functional name=functional, built=True>>


In [6]:
y_predict = model.predict(x_test)
print('예측값: ',  y_predict[:5].ravel())
print('실제값: ', y_test[:5].values.ravel())

# 새로운 값으로 예측
new_x_test = [[2015,"준중형",11.8,172,21.0,"가솔린",0,1999,1300,"자동"]]
new_x_test = pd.DataFrame(new_x_test, \
                        columns=['년식', '종류', '연비', '마력', '토크', '연료', '하이브리드', '배기량', '중량', '변속기'])
print(new_x_test)

new_x_test = transformer.transform(new_x_test)
new_predict = model.predict(new_x_test)
print('예측값: ', new_predict.ravel())

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
예측값:  [0. 0. 0. 0. 0.]
실제값:  [1915 1164 2817 2160 1915]
     년식   종류    연비   마력    토크   연료  하이브리드   배기량    중량 변속기
0  2015  준중형  11.8  172  21.0  가솔린      0  1999  1300  자동
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
예측값:  [0.]
